In [1]:
import pandas as pd

# Load the clinical data from the raw data directory
try:
    clinical_df = pd.read_csv('../data/raw/clinical_data.csv')

    # --- STROBE Tracking ---
    # Track 1: Total cases in the database is 6,388 
    print(f"Total cases in VitalDB: {len(clinical_df)}")

    # Track 2: Filter for General Surgery cases (using .str.lower() for robustness)
    # The data uses 'General surgery' 
    gs_df = clinical_df[clinical_df['department'].str.lower() == 'general surgery'].copy()
    print(f"General Surgery cases: {len(gs_df)}")

    # Track 3: Filter for cases with essential data (preoperative creatinine and opend time)
    initial_cohort_df = gs_df.dropna(subset=['preop_cr', 'opend'])
    print(f"Cases after removing missing essential data: {len(initial_cohort_df)}")

    # Display the first few rows of your initial cohort
    print("\nInitial Cohort DataFrame Head:")
    display(initial_cohort_df.head())

except FileNotFoundError:
    print("Error: 'clinical_data.csv' not found. Make sure it is in the 'data/raw/' directory.")

Total cases in VitalDB: 6388
General Surgery cases: 4930
Cases after removing missing essential data: 4728

Initial Cohort DataFrame Head:


,caseid,subjectid,casestart,caseend,anestart,aneend,opstart,opend,adm,dis,...,intraop_colloid,intraop_ppf,intraop_mdz,intraop_ftn,intraop_rocu,intraop_vecu,intraop_eph,intraop_phe,intraop_epi,intraop_ca
0,1,5955,0,11542,-552,10848.0,1668,10368,-236220,627780,...,0,120,0.0,100,70,0,10,0,0,0
1,2,2487,0,15741,-1039,14921.0,1721,14621,-221160,1506840,...,0,150,0.0,0,100,0,20,0,0,0
2,3,2861,0,4394,-590,4210.0,1090,3010,-218640,40560,...,0,0,0.0,0,50,0,0,0,0,0
3,4,1903,0,20990,-778,20222.0,2522,17822,-201120,576480,...,0,80,0.0,100,100,0,50,0,0,0
4,5,4416,0,21531,-1009,22391.0,2591,20291,-67560,3734040,...,0,0,0.0,0,160,0,10,900,0,2100


In [2]:
# --- Assess Preoperative Variable Completeness ---

# Calculate the percentage of missing values for each column
missing_percentages = initial_cohort_df.isnull().sum() / len(initial_cohort_df) * 100

# Filter to show only columns that have missing values, and sort them
# This makes the output easier to read.
print("Missing value percentages for preoperative variables (>0%):")
print(missing_percentages[missing_percentages > 0].sort_values(ascending=False))

Missing value percentages for preoperative variables (>0%):
dltubesize             99.640440
cline2                 98.963621
lmasize                98.286802
aline2                 98.202200
preop_be               91.878173
preop_hco3             91.857022
preop_sao2             91.857022
preop_pao2             91.751269
preop_paco2            91.751269
preop_ph               91.666667
cline1                 84.115905
iv2                    83.375635
aline1                 57.212352
intraop_uo             44.860406
intraop_ebl            37.267343
cormack                13.832487
tubesize               10.131134
airway                  7.127750
intraop_crystalloid     6.450931
preop_k                 4.695431
preop_na                4.695431
position                3.468697
preop_aptt              1.290186
iv1                     1.184433
preop_pt                0.930626
preop_gluc              0.676819
preop_plt               0.380711
preop_hb                0.232657
preop_ast       

In [3]:
import vitaldb
import numpy as np

# The name for the PLETH waveform track
pleth_track_name = 'SNUADC/PLETH'

print(f"Using vitaldb.find_cases() to find all cases with the '{pleth_track_name}' waveform...")

# 1. Find all cases in the entire dataset that have the PLETH waveform 
cases_with_pleth = vitaldb.find_cases([pleth_track_name])
print(f"Total cases in VitalDB with '{pleth_track_name}': {len(cases_with_pleth)}")

# 2. Get the case IDs from your specific cohort
cohort_case_ids = initial_cohort_df['caseid'].tolist()

# 3. Find the intersection between the two lists to see which of YOUR cases have the waveform
# Using sets is a very fast way to do this
valid_cases_in_cohort = np.intersect1d(cases_with_pleth, cohort_case_ids)

# 4. Report the results
print(f"Found {len(valid_cases_in_cohort)} cases with '{pleth_track_name}' out of {len(cohort_case_ids)} total cases in your cohort.")
availability_percent = (len(valid_cases_in_cohort) / len(cohort_case_ids)) * 100
print(f"Waveform Availability in Cohort: {availability_percent:.2f}%")

# You can now create a final cohort dataframe that only includes cases with the required waveform
final_cohort_with_waveform = initial_cohort_df[initial_cohort_df['caseid'].isin(valid_cases_in_cohort)].copy()

print(f"\nYour final cohort with available waveform data now has {len(final_cohort_with_waveform)} cases.")

Using vitaldb.find_cases() to find all cases with the 'SNUADC/PLETH' waveform...
Total cases in VitalDB with 'SNUADC/PLETH': 6157
Found 4581 cases with 'SNUADC/PLETH' out of 4728 total cases in your cohort.
Waveform Availability in Cohort: 96.89%

Your final cohort with available waveform data now has 4581 cases.


In [4]:
import pandas as pd

# --- Phase 3: Finalize Patient Cohort & Exclude based on Lab Availability ---

# Load the lab results data first, as we'll need it for filtering
lab_df = pd.read_csv('../data/raw/lab_data.csv')
cr_labs = lab_df[lab_df['name'] == 'cr']

# Start with the cohort that has the required waveform data
cohort_df = final_cohort_with_waveform.copy()

# Track 4: Exclude pre-existing severe kidney disease (preop_cr > 4.0)
cohort_df = cohort_df[cohort_df['preop_cr'] <= 4.0].copy()
print(f"Cases after excluding severe kidney disease: {len(cohort_df)}")

# Ensure sample independence by selecting only one surgery per patient first
# This is critical for statistical validity
independent_cohort_df = cohort_df.groupby('subjectid').sample(n=1, random_state=42)
print(f"Cohort size after ensuring sample independence: {len(independent_cohort_df)}")

# ------------------------------------------------------------------------------------
# NEW - Track 5: Exclude patients without outcome data (postoperative labs)
print("\nFiltering cohort to include only patients with postoperative creatinine labs...")

# To do this efficiently, we first find which caseids have the required labs
# Merge cohort opend times with creatinine labs
merged_labs = pd.merge(independent_cohort_df[['caseid', 'opend']], cr_labs, on='caseid')

# Filter for labs taken between the end of the operation and 7 days after
postop_labs_7d = merged_labs[
    (merged_labs['dt'] > merged_labs['opend']) &
    (merged_labs['dt'] <= merged_labs['opend'] + (7 * 24 * 3600))
]

# Get the unique set of caseids that have at least one valid post-op lab
valid_caseids_with_labs = set(postop_labs_7d['caseid'])

# Filter the main cohort to keep only the cases in our valid set
final_cohort_df = independent_cohort_df[independent_cohort_df['caseid'].isin(valid_caseids_with_labs)].copy()

print(f"Final study cohort size after excluding patients without post-op labs: {len(final_cohort_df)}")
# ------------------------------------------------------------------------------------

# Display the head of the truly final cohort before we add the labels
print("\nDisplaying head of the final cohort:")
display(final_cohort_df.head())

Cases after excluding severe kidney disease: 4391
Cohort size after ensuring sample independence: 4240

Filtering cohort to include only patients with postoperative creatinine labs...
Final study cohort size after excluding patients without post-op labs: 2566

Displaying head of the final cohort:


,caseid,subjectid,casestart,caseend,anestart,aneend,opstart,opend,adm,dis,...,intraop_colloid,intraop_ppf,intraop_mdz,intraop_ftn,intraop_rocu,intraop_vecu,intraop_eph,intraop_phe,intraop_epi,intraop_ca
3593,3594,1,0,27769,-575,26725.0,2125,26725,-216840,1079160,...,500,120,0.0,0,110,0,0,0,0,0
6197,6198,3,0,10013,-314,9886.0,1786,8986,-219300,471900,...,0,70,0.0,100,60,0,0,0,0,300
3416,3417,4,0,16177,-447,15813.0,1833,15333,-215940,1598460,...,0,0,0.0,0,90,0,0,0,0,0
733,734,5,0,20889,-76,20624.0,4724,19424,-114060,490740,...,0,0,0.0,0,95,0,0,0,0,0
1579,1580,6,0,13925,-1646,13594.0,2194,13294,-219180,558420,...,500,90,0.0,0,110,0,10,130,0,300


In [5]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# --- Engineer the AKI Outcome Label (KDIGO Criteria) ---

# Load the lab results data
print("Loading lab data...")
lab_df = pd.read_csv('../data/raw/lab_data.csv')

# Filter for only creatinine labs to make the search faster
cr_labs = lab_df[lab_df['name'] == 'cr']
print("Lab data loaded and filtered for creatinine.")

# This is the function definition from your project plan
def get_aki_label(row, cr_labs_df):
    """
    Applies KDIGO criteria to determine the AKI label for a single patient row.
    """
    # Find all creatinine labs for the specific caseid
    case_labs = cr_labs_df[cr_labs_df['caseid'] == row['caseid']]
    
    # Filter for labs that occurred after the operation ended
    postop_labs = case_labs[case_labs['dt'] > row['opend']]
    
    if postop_labs.empty:
        return None # This case should have already been excluded, but as a safeguard.
        
    baseline_cr = row['preop_cr']
    
    # Condition A: Increase of >= 0.3 mg/dL within 48 hours 
    labs_48h = postop_labs[postop_labs['dt'] <= row['opend'] + (48 * 3600)]
    condition_A = False
    if not labs_48h.empty:
        max_cr_48h = labs_48h['result'].max()
        if (max_cr_48h - baseline_cr) >= 0.3:
            condition_A = True
            
    # Condition B: Increase of >= 1.5x baseline within 7 days 
    labs_7d = postop_labs[postop_labs['dt'] <= row['opend'] + (7 * 24 * 3600)]
    condition_B = False
    if not labs_7d.empty:
        max_cr_7d = labs_7d['result'].max()
        if (max_cr_7d / baseline_cr) >= 1.5:
            condition_B = True
            
    return 1 if condition_A or condition_B else 0 # 

# Use tqdm to show progress as this can take a moment
tqdm.pandas(desc="Applying AKI criteria")

# Apply the function to each row of the dataframe
final_cohort_df['aki_label'] = final_cohort_df.progress_apply(lambda row: get_aki_label(row, cr_labs), axis=1)

# Report the distribution of the outcome variable
print("\nAKI Labeling Complete.")
print("Distribution of AKI labels:")
print(final_cohort_df['aki_label'].value_counts(dropna=False))

c:\Users\steal\anaconda3\envs\aki_prediction_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading lab data...
Lab data loaded and filtered for creatinine.


Applying AKI criteria: 100%|██████████| 2566/2566 [00:02<00:00, 1050.35it/s]


AKI Labeling Complete.
Distribution of AKI labels:
aki_label
0    2400
1     166
Name: count, dtype: int64


In [6]:
from sklearn.model_selection import train_test_split

# --- Phase 4: Split Data ---

print("Splitting data into training and testing sets...")

# It's good practice to save the final labeled cohort to a file
# This creates a checkpoint so you don't have to re-run all of Phase 1-3
final_cohort_df.to_csv('../data/processed/final_cohort_with_labels.csv', index=False)
print("Saved final labeled cohort to 'data/processed/final_cohort_with_labels.csv'")

# Define your features (X) and your target (y)
X = final_cohort_df.drop('aki_label', axis=1)
y = final_cohort_df['aki_label']

# Perform an 80/20 split
# We use 'stratify=y' to ensure that the proportion of AKI cases (1s and 0s)
# is the same in both the training and testing sets. This is crucial for imbalanced datasets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

print(f"\nData split complete.")
print(f"Training set size: {len(X_train)} cases")
print(f"Testing set size:  {len(X_test)} cases")
print(f"AKI incidence in training set: {y_train.mean()*100:.2f}%")
print(f"AKI incidence in testing set:  {y_test.mean()*100:.2f}%")

Splitting data into training and testing sets...
Saved final labeled cohort to 'data/processed/final_cohort_with_labels.csv'

Data split complete.
Training set size: 2052 cases
Testing set size:  514 cases
AKI incidence in training set: 6.48%
AKI incidence in testing set:  6.42%
